In [13]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [14]:
url_feat_scaled = "https://raw.githubusercontent.com/ricki0313/Pattern_Recognition_final_project_2025/refs/heads/main/W2_Data%20standardization%2C%20dimensionality%20reduction/scaled_customer_features.csv"
df_feat_scaled = pd.read_csv(url_feat_scaled)
df_feat_scaled.head(3)

,Customer ID,Recency_scaled,Frequency_scaled,Monetary_scaled,AvgUnitPrice_scaled,UniqueItems_scaled,AvgQuantPerOrder_scaled
0,12346.0,2.334858,-0.424675,8.359634,-0.100146,-0.708687,60.891574
1,12347.0,-0.905199,0.354080,0.251046,-0.053249,0.486336,-0.028273
2,12348.0,-0.175186,-0.035297,-0.028547,0.037994,-0.462653,0.022638


In [15]:
feature_cols = ['Recency_scaled','Frequency_scaled','Monetary_scaled','AvgUnitPrice_scaled','UniqueItems_scaled','AvgQuantPerOrder_scaled']
X_feat_scaled = df_feat_scaled[feature_cols].values
K_range = range(2, 11)

In [16]:
sil_score = []
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X_feat_scaled)
    score = silhouette_score(X_feat_scaled, labels)
    sil_score.append(score)
print(sil_score)

[0.4458802835717644, 0.47026644965573444, 0.4655942617472153, 0.4660760573515918, 0.4658693553754538, 0.5103669508561414, 0.4695396541234798, 0.5022146860640287, 0.3992942198072263]


在六維標準化資料中，Silhouette Score 在 K=3 時達到 2~10 範圍內的最高值（0.470），顯示三群結構能有效平衡群內緊密度與群間分離度。雖 K=7、K=9 分數略高，但群數過大且缺乏可解釋性，因此本專案採用 K=3。

In [17]:
kmeans = KMeans(n_clusters=3, random_state=42, n_init=50)
labels = kmeans.fit_predict(X_feat_scaled)
df_feat_scaled['cluster'] = labels

In [18]:
df_feat_scaled["cluster"].value_counts()

cluster
0    3181
1    1140
2      18
Name: count, dtype: int64

在六維標準化特徵下進行 K-means（K=3）後，三個客群的規模呈現合理的分布：一般客戶佔 3181 位、高價值客戶佔 1140 位，而極端大量採購的批發型客戶則形成 18 位的小群。相較於 PCA 分群出現的極端不均衡（4321/17/1），六維特徵分群能更準確反映客戶真實行為，並具備清晰的商業語意，因此採用此模型作為最終分群結果。

小結:
PCA 屬於以變異量最大化為目標的線性降維方法，保留的是資料的「變異方向」，因此 PCA 降維後的資料形狀與距離關係與原始空間不同，尤其會受到極端值影響，使中間客群被壓縮，導致大量中間客戶被壓縮至一般客戶。在六維標準化空間直接進行 K-means，能更完整保留客戶行為資訊，分群結果更穩定且具商業語意。